In [34]:
arr = ["zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]


def fun(s):
    s = s.lower().split()
    ans = ""
    for i in range(len(s)):
        for j in range(10):
            if(s[i].count(arr[j])>0):
                ans += str(j)
                break
        if(len(ans)==3):
            break
    return ans

from transformers import pipeline, AutoTokenizer, DefaultDataCollator, AutoModelForQuestionAnswering, TrainingArguments, Trainer
import pandas as pd
from datasets import Dataset

file_path = '~/advanced/nlp.jsonl'
data = pd.read_json(path_or_buf=file_path, lines=True,dtype={'key':'str', 'transcript':'str', 'tool':'str', 'heading':'str', 'targtet':'str'})

model_name = "deepset/roberta-base-squad2"
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [35]:
data = data.rename(columns={'transcript': 'context'})
data = data.drop(columns=['key', 'tool', 'target'])
questions = ["What is the heading" for q in data["context"]]
data["question"] = questions

b = []
r = []
for i in range(len(data)):
    QA_input = {
        'question': 'What is the heading',
        'context': data[i:i+1]["context"].item()
    }
    n = nlp(QA_input)['answer']
    c = data[i:i+1]["heading"].item()
    d = fun(n)
    if(d != c):
        r.append(i)
        print(i, d, c, n)
    else:
        b.append({'answer_start': [data[i:i+1]["context"].item().find(n)], 'text': [n]})
print(len(data))
data = data.drop(r)
data['answers']=b

524  300 Engage target
716 0 095 zero
722  140 air defense turrets
730 30 130 three zero heading
773 0 090 zero
774  155 Deploy surface-to-air missiles
1084  225 urgent message
1505  010 EMP
1683  165 target is a purple commercial aircraft
1777  350 surface-to-air missiles
2144  280 surface-to-air missiles
2730 80 280 eight zero heading
2898 0 095 zero
3151 15 115 one five
3487 10 310 one zero
3500


In [36]:
data = Dataset.from_pandas(data)
data = data.train_test_split(test_size=0.2)

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [37]:
tokenized_data = data.map(preprocess_function, batched=True, remove_columns=data["train"].column_names)

data_collator = DefaultDataCollator()

model = AutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased")


training_args = TrainingArguments(
    output_dir="headingmodel",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()
trainer.save_model('headingmodel')

Map:   0%|          | 0/2788 [00:00<?, ? examples/s]

Map:   0%|          | 0/697 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,0.041266
2,No log,0.036767
3,0.237600,0.036393


In [33]:
arr = ["zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]


file_path = 'nlp.jsonl'
data = pd.read_json(path_or_buf=file_path, lines=True,dtype={'key':'str', 'transcript':'str', 'tool':'str', 'heading':'str', 'targtet':'str'})
question_answerer = pipeline("question-answering", model="headingmodel")
j = 0
for i in range((len(data))):
    question = "What direction to deploy weapon in"
    context = data[i:i+1]["transcript"].item()

    a = question_answerer(question=question, context=context)["answer"]
    b = data[i:i+1]["heading"].item()
    c = fun(a)
    if(c!=b):
        j+=1
        print(i,j,c,b,a)